In [6]:
data["tag"].value_counts()

tag,count
str,u32
"""ОКР""",5158
"""депрессия""",37475
"""тревожное р-во…",10069
"""БАР""",2794
"""ПРЛ""",5219
"""шизофрения""",2055


In [7]:
data["text"].to_list()[:10]

['Всем добрый день!  У меня БАР 2 типа. Начала часто употреблять спиртные напитки, при этом не помню что происходило. Именно из-за этого несколько раз ругалась с мужем (творила всякую дичь, вплоть до звонков бывшему, чего не помню), в последний раз сильно начали ругаться, не помню из-за чего, потом помню лишь чувство страха и бывший появляется у двери нашей квартиры. Муж взял несколько своих вещей и ушёл. Домой не возвращается уже седьмой день, на контакт не идет вообще. Что делать? Не могу ни спать, ни есть, дикие истерики первые 4 дня были вплоть до самоповреждения. Сейчас полегче, но истерики продолжаются (1 раз в 1-2 дня, ближе к ночи). Очень хочу всё исправить, вернуть любимого, неизвестность тревожит, понимаю, что очень накосячила, переосмыслила всю свою жизнь, отношения, нашла миллион косяков со своей стороны, готова всё исправить, начинаю работать над собой, ищу хорошего психиатра (прошлый психиатр упорно назначал антидепрессанты, с которых мне становилось хуже). Помогите пожал

### Уменьшение и балансировка датасета

In [7]:
target_total_count = 25000

# Определите количество уникальных тегов
unique_tags = data['tag'].n_unique()

# Установите целевое количество строк для каждого класса
target_count_per_class = target_total_count // unique_tags

data = data.with_columns((pl.col('text').str.split(' ').list.lengths()).alias('word_count'))

# Функция для выборки подмножества данных для каждого класса по наибольшему количеству слов
def select_top_by_word_count(df: pl.DataFrame, target_count):
    return df.sort('word_count', descending=True).head(target_count)

# # Функция для выборки подмножества данных для каждого класса
# def sample_class(df, target_count):
#     if df.shape[0] > target_count:
#         return df.sample(n=target_count)
#     return df

# Создайте новый сбалансированный датасет
balanced_dfs = []

for tag in data['tag'].unique():
    class_df = data.filter(pl.col('tag') == tag)
    sampled_class_df = select_top_by_word_count(class_df, target_count_per_class)
    balanced_dfs.append(sampled_class_df)

# Объедините все классы в один датасет
balanced_df = pl.concat(balanced_dfs)

# Если суммарное количество строк больше нужного, случайно уменьшите его до нужного размера
if balanced_df.shape[0] > target_total_count:
    balanced_df = balanced_df.sample(n=target_total_count)

balanced_df = balanced_df.sample(fraction=1, shuffle=True)

/tmp/ipykernel_6842/3899369423.py:9: DeprecationWarning: `lengths` is deprecated. It has been renamed to `len`.
  data = data.with_columns((pl.col('text').str.split(' ').list.lengths()).alias('word_count'))


In [8]:
balanced_df["tag"].value_counts()

tag,count
str,u32
"""депрессия""",4166
"""шизофрения""",2055
"""тревожное р-во…",4166
"""ОКР""",4166
"""ПРЛ""",4166
"""БАР""",2794


In [9]:
balanced_df.shape

(21513, 6)

In [11]:
balanced_df["text"].to_list()[:10]

['Добрый день, как я могу к Вам обращаться? Если согласны с условиями демо-консультации, то можем начать. Условия проведения демо-консультацииДобрый день.Предлагаю проведение демо-консультации в свободном формате в течение 3 дней. -Отвечаю в течение дня. -Если от Вас нет ответа в течение суток - закрываю тему.В результате демо-консультации Вы найдете вариант решения Вашего запроса, либо сформулируете новый запрос. Если консультация оказалась для Вас полезной, буду благодарна за отзыв.https://www.b17.ru/my_comments.php?mod=response&id=813789&prt=813789Вы должны предупредить меня, если имеете проблемы со здоровьем психического характера, зависимостях, суицидальном поведении.Демо-консультация поможет начать работу с Вашей проблемой, но не заменит личную терапию длительного характера.',
 ' Возможно, что срывы случаются в следствии приема психотропных средств, ведь прием таких средств просто "подмораживает" психику и никак не влияет на причину дисбаланса психики и в какой то момент "подморо

### Train val test split

In [10]:
tag_counts = balanced_df.groupby('tag').agg(pl.col('tag').count().alias('tag_count'))
balanced_df = balanced_df.join(tag_counts, on='tag', how='left')
balanced_df.head(3)

/tmp/ipykernel_6842/1634387339.py:1: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  tag_counts = balanced_df.groupby('tag').agg(pl.col('tag').count().alias('tag_count'))


url,date,text,tag,source,word_count,tag_count
str,str,str,str,str,u32,u32
"""https://vk.com…","""2023-03-19 22:…","""Вам попадаются…","""депрессия""","""ВК""",87,4166
"""https://www.b1…",""" 3.01.2023-02:…","""Вы имеете в ви…","""ПРЛ""","""forum b17""",26,4166
"""https://vk.com…","""2019-05-02 19:…","""я вижу только …","""шизофрения""","""ВК""",52,2055


### Version 2 (Финальная)

In [5]:
import polars as pl
import numpy as np
import matplotlib.pyplot as plt

pl.Config.set_fmt_str_lengths(100)

In [72]:
data = pl.read_csv("raw/Data_Sources_csv.csv", separator=";")
data.head(5)

url,date,text,tag,source
str,str,str,str,str
"""https://vk.com…","""2023-04-15 19:…","""Всем добрый де…","""БАР""","""ВК"""
"""https://vk.com…","""2023-04-15 19:…","""это на всякий …","""БАР""","""ВК"""
"""https://vk.com…","""2023-04-15 19:…","""Попробуйте усп…","""БАР""","""ВК"""
"""https://vk.com…","""2023-04-15 19:…","""На все вопросы…","""БАР""","""ВК"""
"""https://vk.com…","""2023-04-15 19:…","""Всё к лучшему.…","""БАР""","""ВК"""


Т.к. разрешено оставить не менее 6 классов, уберем "тревожное р-во/невроз", "тревожное р-во/депрессия", "паранойя"

In [73]:
data = data.drop_nulls()
data = data.filter(~pl.col("tag").is_in(["тревожное р-во/невроз", "тревожное р-во/депрессия", "паранойя"]))
data.shape

(62770, 5)

In [74]:
# pl.Config.set_fmt_str_lengths(100)
# data.select(pl.col(["url", "text", "tag"]).shuffle()).head(500)
# data.filter(pl.col("url") == "https://vk.com/id3452919")
# data.filtergroupby("url").count()

### Заводим аггрегационные признаки для дальнейшей фильтрации:
- Количество слов в документа
- Количество сообщений в рамках одного url

In [75]:
data = data.with_columns((pl.col('text').str.split(' ').list.len()).alias('word_count'))
topic_sizes = data.group_by("url").len().rename({"len": "topic_size"})
data = data.join(
    topic_sizes,
    on=['url'],
    how='left',
)

Фильтруем особым образом, не трогая классы ["тревожное р-во", "депрессия"], т.к. их меньше всего представлено. Убираем строки, где сообщения короткие(в них менее вероятно присутствует нужная нам семантика), а так же убираем топики, которые представлены малым числом сообщений. Второе нам пригодится в будущем. 

In [76]:
# data = data.filter(((pl.col("topic_size") == 1) & (pl.col("word_count") < 20)))
# data = data.filter((pl.col("topic_size") > 15) & (~pl.col("tag").is_in(["БАР", "шизофрения"])))
filtered_data = data.with_columns(
    pl.when(pl.col("tag").is_in(["тревожное р-во", "депрессия"]))
    .then(pl.col("word_count") > 20)
    .otherwise(True)
    .alias("filter_condition1")
).filter(pl.col("filter_condition1"))
filtered_data.drop("filter_condition1")

filtered_data = filtered_data.with_columns(
    pl.when(pl.col("tag").is_in(["тревожное р-во", "депрессия"]))
    .then(pl.col("topic_size") > 20)
    .otherwise(True)
    .alias("filter_condition2")
).filter(pl.col("filter_condition2"))
filtered_data.drop("filter_condition2")


url,date,text,tag,source,word_count,topic_size,filter_condition1
str,str,str,str,str,u32,u32,bool
"""https://vk.com…","""2023-04-15 19:…","""Всем добрый де…","""БАР""","""ВК""",169,1,true
"""https://vk.com…","""2023-04-15 19:…","""это на всякий …","""БАР""","""ВК""",4,32,true
"""https://vk.com…","""2023-04-15 19:…","""Попробуйте усп…","""БАР""","""ВК""",28,126,true
"""https://vk.com…","""2023-04-15 19:…","""На все вопросы…","""БАР""","""ВК""",6,14,true
"""https://vk.com…","""2023-04-15 19:…","""Всё к лучшему.…","""БАР""","""ВК""",24,2,true
"""https://vk.com…","""2023-04-15 20:…","""вы серьезно? А…","""БАР""","""ВК""",31,1,true
"""https://vk.com…","""2023-04-16 2:2…","""я не за болезн…","""БАР""","""ВК""",66,2,true
"""https://vk.com…","""2023-04-15 19:…","""Лучше сменить …","""БАР""","""ВК""",12,1,true
"""https://vk.com…","""2023-04-15 19:…","""Препараты, АД,…","""БАР""","""ВК""",30,1,true


In [77]:
print(filtered_data.shape)
filtered_data["tag"].value_counts()

(35704, 9)


tag,count
str,u32
"""ПРЛ""",5219
"""ОКР""",5158
"""тревожное р-во…",5094
"""шизофрения""",2055
"""депрессия""",15384
"""БАР""",2794


Теперь обрежем датасет до ~20 тыс. строк. Чистим равномерно по каждому классу, в приоритете оставляя тексты с наибольшей длиной

In [84]:
target_total_count = 24000

# количество уникальных тегов
unique_tags = filtered_data['tag'].n_unique()

# целевое количество строк для каждого класса
target_count_per_class = target_total_count // unique_tags

# функция для выборки подмножества данных для каждого класса по наибольшему количеству слов
def select_top_by_word_count(df: pl.DataFrame, target_count):
    return df.sort('word_count', descending=True).head(target_count)

balanced_dfs = []

for tag in filtered_data['tag'].unique():
    class_df = filtered_data.filter(pl.col('tag') == tag)
    sampled_class_df = select_top_by_word_count(class_df, target_count_per_class)
    balanced_dfs.append(sampled_class_df)

# объединяем все полученной в один датафрейм
balanced_df = pl.concat(balanced_dfs)

# Если суммарное количество строк больше нужного, случайно уменьшаем до нужного размера
if balanced_df.shape[0] > target_total_count:
    balanced_df = balanced_df.sample(n=target_total_count)

balanced_df = balanced_df.sample(fraction=1, shuffle=True)

Финальное распределение по классам выглядит следующим образом:

In [85]:
balanced_df["tag"].value_counts()

tag,count
str,u32
"""ОКР""",4000
"""депрессия""",4000
"""БАР""",2794
"""шизофрения""",2055
"""тревожное р-во…",4000
"""ПРЛ""",4000


Уложились в 20 т. строк

In [86]:
balanced_df.shape

(20849, 9)

### Разбиение на train/test

Разбиение проводим равномерно по каждому классу (stratify)

Сохраняем обработанный датасет и идем дальше.

In [87]:
tag_counts = balanced_df.groupby('tag').agg(pl.col('tag').count().alias('tag_count'))
balanced_df = balanced_df.join(tag_counts, on='tag', how='left')
balanced_df.head(3)

from sklearn.model_selection import train_test_split

X = balanced_df.select(["url", "date", "text", "source", "word_count", "topic_size", "tag_count"])
y = balanced_df.select("tag")

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=balanced_df["tag_count"].to_numpy())

X_train.write_parquet("preprocessed/v2/X_train.parquet")
y_train.write_parquet("preprocessed/v2/y_train.parquet")
X_test.write_parquet("preprocessed/v2/X_test.parquet")
y_test.write_parquet("preprocessed/v2/y_test.parquet")

/tmp/ipykernel_32357/3178772607.py:1: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  tag_counts = balanced_df.groupby('tag').agg(pl.col('tag').count().alias('tag_count'))
